# Chapter-9 End to End Execution of Whisper model

#### In this notebook, we will use the concepts which we learned in the earlier chapters and try to play with Whisper model in an end-to-end manner.

### Step-1 Load and Run model in Pytorch

In [1]:
# Install prerequisites
!pip install datasets onnx onnxsim onnxruntime transformers torch librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 KB 4.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.2
    Uninstalling huggingface-hub-0.23.2:
      Successfully uninstalled huggingface-hub-0.23.2


In [2]:
# Import the modules
import os
import time
import onnx
import torch
import numpy as np
from pathlib import Path
import onnxruntime as ort
from IPython.display import Audio
from datasets import load_dataset
from typing import Tuple, Optional, Dict, Any, List
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from onnxruntime.quantization import quantize_dynamic, QuantType
from onnxruntime.quantization.shape_inference import quant_pre_process

whisper_model_name = "openai/whisper-tiny.en"

/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
2025-05-16 22:41:23.185540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747415486.091942    1329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN wh

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# load model and processor
processor = WhisperProcessor.from_pretrained(whisper_model_name)
model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)
model.eval()

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          

In [4]:
# load librispeech ASR dataset and read audio files
ds = load_dataset("librispeech_asr", "clean", split="validation", streaming=True, trust_remote_code=True)
sample = next(iter(ds))

# Extract the waveform and sampling rate
audio_array = sample['audio']['array']
sampling_rate = sample['audio']['sampling_rate']

# Play audio from array
Audio(audio_array, rate=sampling_rate)
print(f"Audio text: {sample['text']}")

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

librispeech_asr.py:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

Audio text: HE WAS IN A FEVERED STATE OF MIND OWING TO THE BLIGHT HIS WIFE'S ACTION THREATENED TO CAST UPON HIS ENTIRE FUTURE


In [5]:
# Preprocess input
input_features = processor(sample['audio']["array"], sampling_rate=sample['audio']["sampling_rate"], return_tensors="pt").input_features
print("Shape of input features ", input_features.shape)

# Predict the tokens
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print("Transcription of given audio: " , transcription)

Shape of input features  torch.Size([1, 80, 3000])


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription of given audio:  [" He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future."]


### Step-2 Exporting model to ONNX

![](./extras/Figure-9.4.png)

In [6]:
encoder_model_path = "whisper_encoder.onnx"
decoder_model_init_path = "whisper_decoder_init.onnx"
decoder_model_step_path = "whisper_decoder_step.onnx"
onnx_opset = 17

In [7]:
# Export Whisper Encoder model to ONNX.

# Dummy input for encoder model: (batch_size=1, feature_dim=80, time_steps=3000)
dummy_input = torch.randn(1, 80, 3000)
dummy_attention_mask = torch.ones_like(dummy_input)

# Export encoder
torch.onnx.export(
    model.model.encoder,
    (dummy_input, dummy_attention_mask),
    encoder_model_path,
    input_names=["input_features", "attention_mask"],
    output_names=["encoder_output"],
    dynamic_axes={
        "input_features": {0: "batch_size", 2: "time_steps"},
        "attention_mask": {0: "batch_size", 2: "time_steps"},
        "encoder_output": {0: "batch_size", 1: "time_steps_by_2"}
    },
    opset_version=onnx_opset,
)
print(f"Encoder model exported as {encoder_model_path}")

/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:1016: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_features.shape[-1] != expected_seq_length:
/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:553: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):


Encoder model exported as whisper_encoder.onnx


#### Export Whisper Decoder in two variants. One model is used for 1st inference and other model is used for all the subsequent inferences.


In [8]:
# Export Whisper Decoder Init model

class WhisperDecoderInit(torch.nn.Module):
    """Whisper decoder initialization model with projection head."""
    
    def __init__(self, decoder: torch.nn.Module, proj_out: torch.nn.Module):
        super().__init__()
        self.decoder = decoder
        self.proj_out = proj_out

    def forward(
        self, 
        decoder_input_ids: torch.Tensor, 
        encoder_hidden_states: torch.Tensor
    ) -> Tuple[torch.Tensor, ...]:
        """Forward pass for decoder initialization.
        
        Args:
            decoder_input_ids: Input token IDs for decoder
            encoder_hidden_states: Hidden states from encoder
            
        Returns:
            Tuple containing logits and past key values
        """
        output = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_hidden_states,
            use_cache=True,
        )
        lm_logits = self.proj_out(output[0])
        return (lm_logits,) + output.past_key_values


# Extract decoder and projection components
decoder = model.model.decoder
proj_out = model.proj_out

# Model configuration
config = model.config
batch = 1
decoder_seq_len = 1
encoder_seq_len = 1500      # depends on your audio length
hidden_size = config.d_model
num_layers = config.decoder_layers
num_heads = model.config.decoder_attention_heads
head_dim = hidden_size // num_heads

# Create dummy inputs
dummy_decoder_input_ids = torch.randint(
    0, config.vocab_size, (batch, decoder_seq_len)
)    # [batch, 1]
dummy_encoder_hidden_states = torch.rand(
    batch, encoder_seq_len, hidden_size
)    # [batch, 1500, 384]

# Define dynamic axes for inputs and outputs
dynamic_axes = {
    "decoder_input_ids": {0: "batch", 1: "seq_len"},
    "encoder_hidden_states": {0: "batch", 1: "encoder_seq_len"}
}
for i in range(num_layers):
    dynamic_axes[f"past_cross_key_{i}"] = {0: "batch", 2: "enc_seq_by_2"}
    dynamic_axes[f"past_cross_value_{i}"] = {0: "batch", 2: "enc_seq_by_2"}
    dynamic_axes[f"past_self_key_{i}"] = {0: "batch", 2: "past_seq"}
    dynamic_axes[f"past_self_value_{i}"] = {0: "batch", 2: "past_seq"}

# Initialize model
init_model = WhisperDecoderInit(decoder, proj_out)

# Prepare output names
init_model_output_names = []
for i in range(num_layers):
    init_model_output_names.extend([
        f"past_self_key_{i}",
        f"past_self_value_{i}",
        f"past_cross_key_{i}",
        f"past_cross_value_{i}"
    ])

# Export to ONNX
torch.onnx.export(
    init_model,
    (dummy_decoder_input_ids, dummy_encoder_hidden_states),
    decoder_model_init_path,
    input_names=["decoder_input_ids", "encoder_hidden_states"],
    output_names=["logits"] + init_model_output_names,
    dynamic_axes=dynamic_axes,
    opset_version=onnx_opset,
)

print(f"Decoder init model exported as {decoder_model_init_path}")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:1476: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:


Decoder init model exported as whisper_decoder_init.onnx


In [9]:
# Export Whisper Decoder Step model

class WhisperDecoderStep(torch.nn.Module):
    """Whisper decoder step model with projection head for sequential decoding."""
    
    def __init__(self, decoder: torch.nn.Module, proj_out: torch.nn.Module):
        super().__init__()
        self.decoder = decoder
        self.proj_out = proj_out

    def forward(
        self,
        decoder_input_ids: torch.Tensor,
        cache_position: torch.Tensor,
        encoder_hidden_states: torch.Tensor,
        *past_key_values: Tuple[torch.Tensor, ...]
    ) -> Tuple[torch.Tensor, ...]:
        """Forward pass for decoder step.
        
        Args:
            decoder_input_ids: Input token IDs for current step
            cache_position: Position index for caching
            encoder_hidden_states: Hidden states from encoder
            *past_key_values: Past key-value pairs for attention
            
        Returns:
            Tuple containing logits and updated key values
        """
        # Reformat past_key_values into expected structure
        num_layers = len(past_key_values) // 4
        reformatted_past = []
        
        for i in range(num_layers):
            layer_past = (
                past_key_values[4 * i],     # self_key
                past_key_values[4 * i + 1], # self_value
                past_key_values[4 * i + 2], # cross_key
                past_key_values[4 * i + 3]  # cross_value
            )
            reformatted_past.append(layer_past)

        outputs = self.decoder(
            input_ids=decoder_input_ids,
            past_key_values=reformatted_past,
            use_cache=True,
            cache_position=cache_position,
            encoder_hidden_states=encoder_hidden_states,
        )
        
        lm_logits = self.proj_out(outputs[0])
        return (lm_logits,) + outputs.past_key_values


def create_dummy_past_key_values(
    batch: int,
    num_layers: int,
    num_heads: int,
    decoder_seq_len: int,
    encoder_seq_len: int,
    head_dim: int
) -> List[torch.Tensor]:
    """Create dummy past key values for ONNX export.
    
    Args:
        batch: Batch size
        num_layers: Number of decoder layers
        num_heads: Number of attention heads
        decoder_seq_len: Length of decoder sequence
        encoder_seq_len: Length of encoder sequence
        head_dim: Dimension of each attention head
        
    Returns:
        List of dummy tensors for past key values
    """
    dummy_kvs = []

    self_attn_kv_shape = (batch, num_heads, decoder_seq_len, head_dim)
    cross_attn_kv_shape = (batch, num_heads, encoder_seq_len, head_dim)
    for _ in range(num_layers):
        # Self attention keys/values
        dummy_kvs.append(torch.rand(*self_attn_kv_shape))  # self_key
        dummy_kvs.append(torch.rand(*self_attn_kv_shape))  # self_value
        # Cross attention keys/values
        dummy_kvs.append(torch.rand(*cross_attn_kv_shape))  # cross_key
        dummy_kvs.append(torch.rand(*cross_attn_kv_shape))  # cross_value
    
    return dummy_kvs


# Create dummy inputs
dummy_decoder_input_ids = torch.randint(0, config.vocab_size, (batch, decoder_seq_len))
dummy_encoder_hidden_states = torch.rand(batch, encoder_seq_len, hidden_size)
dummy_cache_position = torch.tensor([0], dtype=torch.long)
dummy_past_key_values = create_dummy_past_key_values(
    batch, num_layers, num_heads, decoder_seq_len, encoder_seq_len, head_dim
)

# Prepare dynamic axes
dynamic_axes = {
    "decoder_input_ids": {0: "batch", 1: "seq_len"},
    "encoder_hidden_states": {0: "batch", 1: "encoder_seq_len"}
}

# Add dynamic axes for past key values
for name in init_model_output_names:
    if "cross" in name:
        dynamic_axes[name] = {0: "batch", 2: "enc_seq_by_2"}
    else:
        dynamic_axes[name] = {0: "batch", 2: "past_seq"}

# Prepare output names
step_output_names = []
for i in range(num_layers):
    step_output_names.extend([
        f"present_self_key_{i}",
        f"present_self_value_{i}",
        f"present_cross_key_{i}",
        f"present_cross_value_{i}"
    ])

# Initialize and export model
step_model = WhisperDecoderStep(model.model.decoder, model.proj_out)

torch.onnx.export(
    step_model,
    (dummy_decoder_input_ids, dummy_cache_position, dummy_encoder_hidden_states, *dummy_past_key_values),
    decoder_model_step_path,
    input_names=["decoder_input_ids", "cache_position", "encoder_hidden_states"] + init_model_output_names,
    output_names=["logits"] + step_output_names,
    dynamic_axes=dynamic_axes,
    opset_version=onnx_opset,
)

print(f"Decoder step model exported as {decoder_model_step_path}")

/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/transformers/cache_utils.py:451: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/mnt/d/Meet/Company/Orange Eva Publication/Jupyter Notebook/onnx_env/lib/python3.10/site-packages/transformers/cache_utils.py:435: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  len(self.key_cache[layer_idx]) == 0


Decoder step model exported as whisper_decoder_step.onnx


In [10]:
# Optimize models using ONNX Simplifier

!python -m onnxsim whisper_encoder.onnx whisper_encoder_opt.onnx
!python -m onnxsim whisper_decoder_init.onnx whisper_decoder_init_opt.onnx
!python -m onnxsim whiasper_decoder_step.onnx whisper_decoder_step_opt.onnx

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                    ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add                │ 35             │ 35               │
│ Cast               │ 8              │ 0                │
│ Concat             │ 16             │ 4                │
│ Constant           │ 165            │ 78               │
│ Conv               │ 2              │ 2                │
│ Div                │ 10             │ 6                │
│ Erf                │ 6              │ 6                │
│ Gather             │ 8              │ 4                │
│ LayerNormalization │ 9              │ 9                │
│ MatMul             │ 32             │ 32               │
│ Mul                │ 20             │ 20               │
│ Reshape            │ 16             │ 16               │
│ Shape              │ 12             │ 4                │
│ Slice  

In [11]:
def run_whisper(encoder_path, decoder_init_path, decoder_step_path, waveform, processor, num_iters=100):
    # Load all three ONNX model
    encoder_sess = ort.InferenceSession(encoder_path)
    decoder_init_sess = ort.InferenceSession(decoder_init_path)
    decoder_step_sess = ort.InferenceSession(decoder_step_path)

    # Convert to log-mel spectrogram
    inputs = processor(waveform, sampling_rate=16000, return_tensors="np")
    input_features = inputs["input_features"]       # (1, 80, time)
    print("Encoder input shape: ", input_features.shape)
          
    # Run encoder model
    encoder_hidden_states = encoder_sess.run(["encoder_output"], 
                                             {"input_features": input_features})[0]  
    # (1, seq_len, hidden_size)
    print("Encoder output shape: ", encoder_hidden_states.shape)

    # Initial input for decoder
    SOT_TOKEN = 50257           # Start of transcript token
    EOS_TOKEN = 50256           # End of sentence token
    decoder_input_ids = np.array([[SOT_TOKEN]], dtype=np.int64)
    
    # Run decoder_init model
    inputs = {
        "decoder_input_ids": decoder_input_ids,
        "encoder_hidden_states": encoder_hidden_states,
    }
    init_outputs = decoder_init_sess.run(None, inputs)
    
    # Decode the decoder_init output
    logits = init_outputs[0]
    print("Logits shape: ", logits.shape)
    generated_ids = [int(np.argmax(logits[0, -1]))]

    # Fetch the KV caches which are generated
    past_self_key = init_outputs[1::4]
    past_self_value = init_outputs[2::4]
    past_cross_key = init_outputs[3::4]
    past_cross_value = init_outputs[4::4]
    
    # Iteratively run decoder_step model
    for step in range(num_iters):
        decoder_input_ids = np.array([[generated_ids[-1]]], dtype=np.int64)
    
        step_inputs = {
            "decoder_input_ids": decoder_input_ids,
            "cache_position": np.array([step + 1], dtype=np.int64),
        }
    
        # Flatten past_kv to feed as input
        for i in range(num_layers):
            step_inputs[f"past_self_key_{i}"] = past_self_key[i]
            step_inputs[f"past_self_value_{i}"] = past_self_value[i]
            step_inputs[f"past_cross_key_{i}"] = past_cross_key[i]
            step_inputs[f"past_cross_value_{i}"] = past_cross_value[i]

        # Run decoder step model
        step_outputs = decoder_step_sess.run(None, step_inputs)

        # Get the predicted token
        logits = step_outputs[0]
        next_token = int(np.argmax(logits[0, -1]))

        # Stop the loop if "End of Sentence" token is found.
        if next_token == EOS_TOKEN:
            break
    
        generated_ids.append(next_token)
    
        # Update KV cache for next step
        past_self_key = step_outputs[1::4]
        past_self_value = step_outputs[2::4]
        past_cross_key = step_outputs[3::4]
        past_cross_value = step_outputs[4::4]
    
    
    # Display the combined prediction from decoder_init and decoder_step.
    print("Generated token IDs:", generated_ids)
    transcription = processor.batch_decode([generated_ids], skip_special_tokens=True)
    print("Text transcription: ", transcription)

    return transcription

In [12]:
# Load and process audio
waveform = sample['audio']['array']

# Run the optimized model in FP32 precision
run_whisper("whisper_encoder_opt.onnx", "whisper_decoder_init_opt.onnx", "whisper_decoder_step_opt.onnx", waveform, processor, num_iters=100)

Encoder input shape:  (1, 80, 3000)
Encoder output shape:  (1, 1500, 384)
Logits shape:  (1, 1, 51864)
Generated token IDs: [50362, 679, 373, 287, 257, 730, 21917, 1181, 286, 2000, 11, 27120, 284, 262, 42514, 465, 3656, 338, 2223, 8556, 284, 3350, 2402, 465, 2104, 2003, 13]
Text transcription:  [" He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future."]


[" He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future."]

### Step-3 Quantizing models

In [13]:
# Apply Dynamic quantization as Whisper is Transformer based model.

def apply_dynamic_quantization(fp32_path):
    root, ext = os.path.splitext(fp32_path)
    fp32_path_preproc = f"{root}_preproc{ext}"        
    int8_path_dynamic_quant = f"{root}_int8_dynamic_quant{ext}"
    
    # Firstly, apply shape inference and onnxruntime model optimization before quantizing the model.
    quant_pre_process(fp32_path, fp32_path_preproc, skip_symbolic_shape=True)

    # Apply dynamic quantization
    quantize_dynamic(
        model_input=fp32_path_preproc,          # Input ONNX model
        model_output=int8_path_dynamic_quant,   # Output quantized model
        weight_type=QuantType.QUInt8            # Quantize only weights to uint8, activations will be quantize during runtime
    )
    
    print(f"Dynamic Quantized model saved at: {int8_path_dynamic_quant}")
    
    # Compare model sizes
    fp32_size = os.path.getsize(fp32_path) / 1024 / 1024
    dynamic_quant_size = os.path.getsize(int8_path_dynamic_quant) / 1024 / 1024
    
    print(f"FP32 Model Size: {fp32_size:.2f} MB")
    print(f"Dynamic Quantized Model Size: {dynamic_quant_size:.2f} MB")

    return int8_path_dynamic_quant

In [14]:
# Quantize encoder model
encoder_int8 = apply_dynamic_quantization("./whisper_encoder_opt.onnx")

# Quantize decoder init model
decoder_init_int8 = apply_dynamic_quantization("./whisper_decoder_init_opt.onnx")

# Quantize decoder step model
decoder_step_int8 = apply_dynamic_quantization("./whisper_decoder_step_opt.onnx")

Dynamic Quantized model saved at: ./whisper_encoder_opt_int8_dynamic_quant.onnx
FP32 Model Size: 31.35 MB
Static Quantized Model Size: 9.60 MB
Dynamic Quantized model saved at: ./whisper_decoder_init_opt_int8_dynamic_quant.onnx
FP32 Model Size: 188.81 MB
Static Quantized Model Size: 47.41 MB
Dynamic Quantized model saved at: ./whisper_decoder_step_opt_int8_dynamic_quant.onnx
FP32 Model Size: 184.30 MB
Static Quantized Model Size: 46.26 MB


### Step-4 Run quantized models

In [15]:
# Load and process audio
waveform = sample['audio']['array']

# Run the optimized model in FP32 precision
run_whisper(encoder_int8, decoder_init_int8, decoder_step_int8, waveform, processor, num_iters=100)

Encoder input shape:  (1, 80, 3000)
Encoder output shape:  (1, 1500, 384)
Logits shape:  (1, 1, 51864)
Generated token IDs: [50362, 679, 373, 287, 257, 730, 21917, 1181, 286, 2000, 11, 27120, 284, 262, 42514, 465, 3656, 338, 2223, 8556, 284, 3350, 2402, 465, 2104, 2003, 13]
Text transcription:  [" He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future."]


[" He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future."]